In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Question 4**

In [ ]:
!pip install streamlit pyngrok

In [ ]:
%%writefile banking_ai_app.py
import streamlit as st
import tensorflow as tf
import pickle
import json
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from datetime import datetime

# ============= CONFIGURATION =============
MODEL_DIR = '/content/drive/MyDrive/Colab Notebooks/AI_CW_010/Final_Model/'

# ============= LOAD MODEL AND COMPONENTS =============
@st.cache_resource
def load_model_components():
    """Load all model components with caching for better performance"""

    # Load the trained model
    model = tf.keras.models.load_model(f'{MODEL_DIR}hierarchical_3headed_banking_model_complete_V3.keras')

    # Load tokenizer
    with open('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/banking_tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)

    # Load label encoders
    with open('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/Final_Model/label_encoders_3headed_V3.pkl', 'rb') as f:
        label_encoders = pickle.load(f)

    # Load model info
    with open(f'{MODEL_DIR}model_info_3headed_V3.json', 'r') as f:
        model_info = json.load(f)

    return model, tokenizer, label_encoders, model_info

# ============= TEXT PREPROCESSING FUNCTIONS =============

def clean_text(text):
    """Clean and preprocess text (same as training)"""
    if pd.isna(text):
        return ""

    # Convert to lowercase
    text = str(text).lower()

    # Remove special characters but keep spaces and basic punctuation
    text = re.sub(r'[^\w\s.,!?-]', '', text)

    # Remove extra whitespaces
    text = ' '.join(text.split())

    return text

def preprocess_input_text(text, tokenizer, max_length):
    """Preprocess single input text for prediction"""

    # Clean the text
    cleaned_text = clean_text(text)

    # Convert text to sequences
    sequence = tokenizer.texts_to_sequences([cleaned_text])

    # Pad sequences
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    return padded_sequence

# ============= PREDICTION FUNCTIONS =============

def make_prediction(text, model, tokenizer, label_encoders, max_length):
    """Make prediction on input text"""

    try:
        # Preprocess the input text
        processed_text = preprocess_input_text(text, tokenizer, max_length)

        # Make prediction
        predictions = model.predict(processed_text, verbose=0)

        # Extract predictions for each head
        category_probs = predictions[0][0]
        intent_probs = predictions[1][0]
        urgency_probs = predictions[2][0]

        # Get predicted classes and confidences
        category_pred = np.argmax(category_probs)
        intent_pred = np.argmax(intent_probs)
        urgency_pred = np.argmax(urgency_probs)

        category_confidence = category_probs[category_pred]
        intent_confidence = intent_probs[intent_pred]
        urgency_confidence = urgency_probs[urgency_pred]

        # Convert to readable labels
        category_label = label_encoders['category_encoder'].inverse_transform([category_pred])[0]
        intent_label = label_encoders['intent_encoder'].inverse_transform([intent_pred])[0]
        urgency_label = label_encoders['urgency_encoder'].inverse_transform([urgency_pred])[0]

        return {
            'category': {'label': category_label, 'confidence': float(category_confidence)},
            'intent': {'label': intent_label, 'confidence': float(intent_confidence)},
            'urgency': {'label': urgency_label, 'confidence': float(urgency_confidence)},
            'success': True
        }

    except Exception as e:
        return {'success': False, 'error': str(e)}

# ============= HELPER FUNCTIONS =============

def get_confidence_class(confidence):
    """Get CSS class based on confidence level"""
    if confidence >= 0.8:
        return "confidence-high"
    elif confidence >= 0.6:
        return "confidence-medium"
    else:
        return "confidence-low"

def get_urgency_emoji(urgency):
    """Get emoji for urgency level"""
    emoji_map = {
        'Low': '🟢',
        'Medium': '🟡',
        'High': '🔴'
    }
    return emoji_map.get(urgency, '⚪')

def generate_analysis_text(result):
    """Generate detailed analysis text"""
    category = result['category']['label']
    intent = result['intent']['label']
    urgency = result['urgency']['label']

    analysis = f"""

    <strong>Customer Query Analysis:</strong><br><br>
    This customer query has been classified as a <strong>{category}</strong> related inquiry,
    specifically requesting <strong>{intent}</strong>. The urgency level has been assessed as
    <strong>{urgency}</strong>, indicating the priority level for handling this request.

    """

    return analysis

def generate_recommendations(result):
    """Generate action recommendations based on predictions"""
    recommendations = []

    # Category-based recommendations
    category = result['category']['label']
    if 'Card' in category:
        recommendations.append("Route to Card Services department")
    elif 'Account' in category:
        recommendations.append("Route to Account Services department")
    elif 'Transfer' in category:
        recommendations.append("Route to Transfer Services department")
    else:
        recommendations.append("Route to General Support team")

    # Urgency-based recommendations
    urgency = result['urgency']['label']
    if urgency == 'High':
        recommendations.append("⚠️ HIGH PRIORITY - Handle immediately")
        recommendations.append("Escalate to senior support if needed")
    elif urgency == 'Medium':
        recommendations.append("Handle within 2-4 hours")
    else:
        recommendations.append("Handle within standard timeframe")

    # Confidence-based recommendations
    min_confidence = min(
        result['category']['confidence'],
        result['intent']['confidence'],
        result['urgency']['confidence']
    )

    if min_confidence < 0.7:
        recommendations.append("⚠️ Low confidence detected - Consider manual review")

    return recommendations

# ============= STREAMLIT UI =============

def main():
    # Page configuration
    st.set_page_config(
        page_title="Banking AI Customer Service Assistant",
        page_icon="🏦",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    # Custom CSS
    st.markdown("""
    <style>
    .main-header {
        font-size: 2.5rem;
        color: #1f4e79;
        text-align: center;
        margin-bottom: 2rem;
    }
    .prediction-box {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 10px;
        border-left: 5px solid #1f4e79;
    }
    .confidence-high { color: #28a745; font-weight: bold; }
    .confidence-medium { color: #ffc107; font-weight: bold; }
    .confidence-low { color: #dc3545; font-weight: bold; }
    </style>
    """, unsafe_allow_html=True)

    # Main header
    st.markdown('<h1 class="main-header">🏦 Banking AI Customer Service Assistant</h1>', unsafe_allow_html=True)

    # Load model components
    with st.spinner('Loading AI model components...'):
        model, tokenizer, label_encoders, model_info = load_model_components()

    # Sidebar with model info
    with st.sidebar:
        st.header("📊 Model Information")
        st.write(f"**Model Version:** V3")
        st.write(f"**Vocabulary Size:** {model_info.get('vocab_size', 'N/A')}")
        st.write(f"**Max Sequence Length:** {model_info.get('max_length', 'N/A')}")
        st.write(f"**Training Samples:** {model_info.get('train_samples', 'N/A')}")

        st.header("🎯 Model Capabilities")
        st.write("**Category Classification:**")
        st.write("• Card Services")
        st.write("• Account Services")
        st.write("• Transfer Services")
        st.write("• Support Services")

        st.write("**Intent Recognition:**")
        st.write("• 77 different banking intents")

        st.write("**Urgency Detection:**")
        st.write("• Low, Medium, High urgency")

    # Main content area
    col1, col2 = st.columns([2, 1])

    with col1:
        st.header("💬 Customer Query Analysis")

        # Text input
        user_input = st.text_area(
            "Enter customer query:",
            placeholder="Example: I need to check my account balance urgently...",
            height=150,
            help="Enter a banking-related customer query for AI analysis"
        )

        # Analysis button
        if st.button("🔍 Analyze Query", type="primary", use_container_width=True):
            if user_input.strip():
                with st.spinner('Analyzing customer query...'):
                    # Make prediction
                    result = make_prediction(
                        user_input,
                        model,
                        tokenizer,
                        label_encoders,
                        model_info['max_length']
                    )

                if result['success']:
                    # Display results
                    st.success("Analysis completed successfully!")

                    # Create results display
                    col_cat, col_intent, col_urgency = st.columns(3)

                    with col_cat:
                        st.markdown("### 📂 Category")
                        confidence = result['category']['confidence']
                        conf_class = get_confidence_class(confidence)
                        st.markdown(f"**{result['category']['label']}**")
                        st.markdown(f'<span class="{conf_class}">Confidence: {confidence:.1%}</span>',
                                  unsafe_allow_html=True)

                    with col_intent:
                        st.markdown("### 🎯 Intent")
                        confidence = result['intent']['confidence']
                        conf_class = get_confidence_class(confidence)
                        st.markdown(f"**{result['intent']['label']}**")
                        st.markdown(f'<span class="{conf_class}">Confidence: {confidence:.1%}</span>',
                                  unsafe_allow_html=True)

                    with col_urgency:
                        st.markdown("### ⚡ Urgency")
                        confidence = result['urgency']['confidence']
                        conf_class = get_confidence_class(confidence)
                        urgency_emoji = get_urgency_emoji(result['urgency']['label'])
                        st.markdown(f"**{urgency_emoji} {result['urgency']['label']}**")
                        st.markdown(f'<span class="{conf_class}">Confidence: {confidence:.1%}</span>',
                                  unsafe_allow_html=True)

                    # Detailed analysis
                    st.markdown("---")
                    st.markdown("### 📋 Detailed Analysis")

                    analysis_text = generate_analysis_text(result)
                    #st.markdown(f'<div class="prediction-box">{analysis_text}</div>',
                              #unsafe_allow_html=True)
                    st.markdown(analysis_text,unsafe_allow_html=True)


                    # Recommendations
                    recommendations = generate_recommendations(result)
                    st.markdown("### 💡 Recommended Actions")
                    for rec in recommendations:
                        st.write(f"• {rec}")

                else:
                    st.error(f"Error during analysis: {result['error']}")
            else:
                st.warning("Please enter a customer query to analyze.")

    with col2:
        st.header("📈 Performance Metrics")

        #  performance metrics
        metrics_data = {
            'Category': 0.87,
            'Intent': 0.90,
            'Urgency': 0.75,
            'Overall': 0.83
        }

        for metric, score in metrics_data.items():
            st.metric(f"{metric} Accuracy", f"{score:.1%}")

        st.markdown("---")

        # Example queries
        st.header("💡 Example Queries")
        example_queries = [
            "I urgently need to transfer money to my friend",
            "My card is not working at the ATM",
            "Can you help me check my account balance?",
            "I want to apply for a new credit card",
            "There's an unauthorized transaction on my account"
        ]

        for i, example in enumerate(example_queries):
            if st.button(f"Try Example {i+1}", key=f"example_{i}"):
                st.text_area("Query:", value=example, key=f"example_text_{i}")

# ============= RUN THE APP =============
if __name__ == "__main__":
    main()

Overwriting banking_ai_app.py


In [ ]:
# ============= LAUNCH THE STREAMLIT APP =============

import subprocess
import threading
import time
from pyngrok import ngrok


# 1. Setup ngrok authentication

ngrok_authtoken = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"  #Replace with actual token
ngrok.set_auth_token(ngrok_authtoken)

print(" Setting up Banking AI Streamlit application...")

# 2. Function to run Streamlit
def run_streamlit():
    """Run Streamlit app in background"""
    subprocess.run([
        'streamlit', 'run', 'banking_ai_app.py',
        '--server.port=8501',
        '--server.headless=True',
        '--server.enableCORS=False',
        '--server.enableXsrfProtection=False'
    ])

# 3. Start Streamlit in a separate thread
print(" Starting Streamlit server...")
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.daemon = True
streamlit_thread.start()

# 4. Wait for Streamlit to initialize
print(" Waiting for Streamlit to start...")
time.sleep(15)  # Give it time to load the model

# 5. Create public tunnel with ngrok
print(" Creating public tunnel...")
try:
    public_url = ngrok.connect(8501)
    print("\n" + "="*60)
    print("✅ BANKING AI WEB APPLICATION IS LIVE!")
    print("="*60)
    print(f"Public URL: {public_url}")
    print(f"Access your app: {public_url}")
    print("="*60)

    print("\n⚠️  Keep this cell running to maintain the public URL")
    print("🛑 Stop this cell to shut down the app")

    # Keep the application running
    try:
        while True:
            time.sleep(30)
            print(" App is running... (Press stop to shutdown)")
    except KeyboardInterrupt:
        print("\n🛑 Shutting down application...")
        ngrok.disconnect(public_url)
        print("✅ Application stopped successfully")

except Exception as e:
    print(f"❌ Error starting application: {e}")


 Setting up Banking AI Streamlit application...
 Starting Streamlit server...
 Waiting for Streamlit to start...
 Creating public tunnel...

✅ BANKING AI WEB APPLICATION IS LIVE!
Public URL: NgrokTunnel: "https://4c4318b4d072.ngrok-free.app" -> "http://localhost:8501"
Access your app: NgrokTunnel: "https://4c4318b4d072.ngrok-free.app" -> "http://localhost:8501"

⚠️  Keep this cell running to maintain the public URL
🛑 Stop this cell to shut down the app
 App is running... (Press stop to shutdown)
 App is running... (Press stop to shutdown)
 App is running... (Press stop to shutdown)

🛑 Shutting down application...
❌ Error starting application: ngrok client exception, URLError: [Errno 111] Connection refused


# **Question 5**

# Adding Explainability Feature to the Model

In [ ]:
!pip install streamlit pyngrok lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=762a702c9d552304a02d0f2d9b282c4a031450cf84a09a57f41bb9d633028ea4
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [ ]:
!pip install lime

In [ ]:
import lime
import lime.lime_text
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
%%writefile explainable_banking_ai_app.py
import streamlit as st
import tensorflow as tf
import pickle
import json
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from datetime import datetime
import lime
import lime.lime_text
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# ============= CONFIGURATION =============
MODEL_DIR = '/content/drive/MyDrive/Colab Notebooks/AI_CW_010/Final_Model/'

# ============= LIME EXPLAINER CLASS =============

class StreamlitBankingExplainer:
    """LIME Explainer integrated for Streamlit"""

    def __init__(self, model, tokenizer, label_encoders, max_length):
        self.model = model
        self.tokenizer = tokenizer
        self.label_encoders = label_encoders
        self.max_length = max_length

        # Initialize LIME
        self.lime_explainer = lime.lime_text.LimeTextExplainer(
            class_names=None,
            split_expression=r'\W+',
            bow=False,
            random_state=42
        )

    def predict_probabilities(self, texts):
        """Prediction function for LIME"""
        processed_texts = []

        for text in texts:
            text = str(text).lower()
            sequence = self.tokenizer.texts_to_sequences([text])
            padded = pad_sequences(sequence, maxlen=self.max_length, padding='post')
            processed_texts.append(padded[0])

        processed_array = np.array(processed_texts)
        predictions = self.model.predict(processed_array, verbose=0)
        return predictions

    def explain_prediction(self, text, head='category', num_features=8):
        """Generate LIME explanation for specific head"""

        head_info = {
            'category': {'index': 0, 'encoder': 'category_encoder'},
            'intent': {'index': 1, 'encoder': 'intent_encoder'},
            'urgency': {'index': 2, 'encoder': 'urgency_encoder'}
        }

        encoder = self.label_encoders[head_info[head]['encoder']]
        self.lime_explainer.class_names = list(encoder.classes_)

        def predict_fn(texts):
            all_preds = self.predict_probabilities(texts)
            head_preds = all_preds[head_info[head]['index']]
            return head_preds

        explanation = self.lime_explainer.explain_instance(
            text, predict_fn, num_features=num_features, num_samples=500
        )

        return explanation

# ============= LOAD MODEL AND COMPONENTS =============
@st.cache_resource
def load_model_components():
    """Load all model components with caching for better performance"""

    # Load the trained model
    model = tf.keras.models.load_model(f'{MODEL_DIR}hierarchical_3headed_banking_model_complete_V3.keras')

    # Load tokenizer
    with open('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/banking_tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)

    # Load label encoders
    with open('/content/drive/MyDrive/Colab Notebooks/AI_CW_010/Final_Model/label_encoders_3headed_V3.pkl', 'rb') as f:
        label_encoders = pickle.load(f)

    # Load model info
    with open(f'{MODEL_DIR}model_info_3headed_V3.json', 'r') as f:
        model_info = json.load(f)

    # Initialize explainer
    explainer = StreamlitBankingExplainer(model, tokenizer, label_encoders, model_info['max_length'])

    return model, tokenizer, label_encoders, model_info, explainer

# ============= TEXT PREPROCESSING FUNCTIONS =============

def clean_text(text):
    """Clean and preprocess text (same as training)"""
    if pd.isna(text):
        return ""

    text = str(text).lower()
    text = re.sub(r'[^\w\s.,!?-]', '', text)
    text = ' '.join(text.split())

    return text

def preprocess_input_text(text, tokenizer, max_length):
    """Preprocess single input text for prediction"""
    cleaned_text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

# ============= PREDICTION FUNCTIONS =============

def make_prediction_with_explanations(text, model, tokenizer, label_encoders, max_length, explainer):
    """Make prediction with LIME explanations"""

    try:
        # Regular prediction
        processed_text = preprocess_input_text(text, tokenizer, max_length)
        predictions = model.predict(processed_text, verbose=0)

        # Extract predictions for each head
        category_probs = predictions[0][0]
        intent_probs = predictions[1][0]
        urgency_probs = predictions[2][0]

        # Get predicted classes and confidences
        category_pred = np.argmax(category_probs)
        intent_pred = np.argmax(intent_probs)
        urgency_pred = np.argmax(urgency_probs)

        category_confidence = category_probs[category_pred]
        intent_confidence = intent_probs[intent_pred]
        urgency_confidence = urgency_probs[urgency_pred]

        # Convert to readable labels
        category_label = label_encoders['category_encoder'].inverse_transform([category_pred])[0]
        intent_label = label_encoders['intent_encoder'].inverse_transform([intent_pred])[0]
        urgency_label = label_encoders['urgency_encoder'].inverse_transform([urgency_pred])[0]

        # Generate LIME explanations
        with st.spinner('Generating explanations...'):
            category_explanation = explainer.explain_prediction(text, head='category', num_features=8)
            intent_explanation = explainer.explain_prediction(text, head='intent', num_features=8)
            urgency_explanation = explainer.explain_prediction(text, head='urgency', num_features=8)

        return {
            'category': {'label': category_label, 'confidence': float(category_confidence)},
            'intent': {'label': intent_label, 'confidence': float(intent_confidence)},
            'urgency': {'label': urgency_label, 'confidence': float(urgency_confidence)},
            'explanations': {
                'category': category_explanation,
                'intent': intent_explanation,
                'urgency': urgency_explanation
            },
            'success': True
        }

    except Exception as e:
        return {'success': False, 'error': str(e)}

# ============= EXPLANATION VISUALIZATION FUNCTIONS =============

def create_explanation_chart(explanation, head_name):
    """Create interactive explanation chart using Plotly"""

    exp_data = explanation.as_list()
    if not exp_data:
        return None

    words = [item[0] for item in exp_data]
    importances = [item[1] for item in exp_data]

    # Create color map
    colors = ['#28a745' if imp > 0 else '#dc3545' for imp in importances]

    fig = go.Figure(data=[
        go.Bar(
            y=words,
            x=importances,
            orientation='h',
            marker=dict(color=colors),
            text=[f'{imp:.3f}' for imp in importances],
            textposition='outside',
            hovertemplate='<b>%{y}</b><br>Importance: %{x:.3f}<extra></extra>'
        )
    ])

    fig.update_layout(
        title=f'{head_name} Word Importance',
        xaxis_title='LIME Importance Score',
        yaxis_title='Words',
        height=400,
        margin=dict(l=100, r=50, t=50, b=50)
    )

    return fig

def highlight_text_with_explanations(text, explanation):
    """Create highlighted text based on LIME explanation"""

    exp_data = explanation.as_list()
    exp_dict = {word: importance for word, importance in exp_data}

    words = text.split()
    highlighted_words = []

    for word in words:
        clean_word = re.sub(r'[^\w]', '', word.lower())

        if clean_word in exp_dict:
            importance = exp_dict[clean_word]
            if importance > 0:
                # Positive importance - green
                intensity = min(abs(importance) * 2, 1.0)
                highlighted_words.append(f'<span style="background-color: rgba(40, 167, 69, {intensity}); padding: 2px 4px; border-radius: 3px; margin: 1px;">{word}</span>')
            else:
                # Negative importance - red
                intensity = min(abs(importance) * 2, 1.0)
                highlighted_words.append(f'<span style="background-color: rgba(220, 53, 69, {intensity}); padding: 2px 4px; border-radius: 3px; margin: 1px;">{word}</span>')
        else:
            highlighted_words.append(word)

    return ' '.join(highlighted_words)

# ============= HELPER FUNCTIONS =============

def get_confidence_class(confidence):
    """Get CSS class based on confidence level"""
    if confidence >= 0.8:
        return "confidence-high"
    elif confidence >= 0.6:
        return "confidence-medium"
    else:
        return "confidence-low"

def get_urgency_emoji(urgency):
    """Get emoji for urgency level"""
    emoji_map = {
        'Low': '🟢',
        'Medium': '🟡',
        'High': '🔴'
    }
    return emoji_map.get(urgency, '⚪')


#def generate_recommendations(result):
    #"""Generate action recommendations based on predictions"""
    #recommendations = []

    # Category-based recommendations
    #category = result['category']['label']
    #if 'Card' in category:
        #recommendations.append("Route to Card Services department")
    #elif 'Account' in category:
        #recommendations.append("Route to Account Services department")
    #elif 'Transfer' in category:
        #recommendations.append("Route to Transfer Services department")
    #else:
        #recommendations.append("Route to General Support team")

    # Urgency-based recommendations
    #urgency = result['urgency']['label']
    #if urgency == 'High':
        #recommendations.append("⚠️ HIGH PRIORITY - Handle immediately")
        #recommendations.append("Escalate to senior support if needed")
    #elif urgency == 'Medium':
        #recommendations.append("Handle within 2-4 hours")
    #else:
        #recommendations.append("Handle within standard timeframe")


# ============= STREAMLIT UI =============

def main():
    # Page configuration
    st.set_page_config(
        page_title="Banking AI Customer Service Assistant",
        page_icon="🏦",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    # Custom CSS
    st.markdown("""
    <style>
    .main-header {
        font-size: 2.5rem;
        color: #1f4e79;
        text-align: center;
        margin-bottom: 2rem;
    }
    .explanation-section {
        background-color: #f8f9fa;
        padding: 1rem;
        border-radius: 10px;
        border-left: 5px solid #007bff;
        margin: 1rem 0;
    }
    .confidence-high { color: #28a745; font-weight: bold; }
    .confidence-medium { color: #ffc107; font-weight: bold; }
    .confidence-low { color: #dc3545; font-weight: bold; }
    .explainability-badge {
        background-color: #17a2b8;
        color: white;
        padding: 0.25rem 0.5rem;
        border-radius: 0.25rem;
        font-size: 0.8rem;
    }
    </style>
    """, unsafe_allow_html=True)

    # Main header with explainability badge
    st.markdown('''
    <h1 class="main-header">
        🏦 Banking AI Customer Service Assistant
        <span class="explainability-badge">XAI Powered</span>
    </h1>
    ''', unsafe_allow_html=True)

    # Load model components
    with st.spinner('Loading AI model and explainability components...'):
        model, tokenizer, label_encoders, model_info, explainer = load_model_components()

    # Sidebar with enhanced info
    with st.sidebar:
        st.header("📊 Model Information")
        st.write(f"**Model Version:** V3 (XAI Enhanced)")
        st.write(f"**Explainability Method:** LIME")
        st.write(f"**Vocabulary Size:** {model_info.get('vocab_size', 'N/A')}")
        st.write(f"**Max Sequence Length:** {model_info.get('max_length', 'N/A')}")

        st.header("🎯 Model Capabilities")
        st.write("**Category Classification:**")
        st.write("• Card Services")
        st.write("• Account Services")
        st.write("• Transfer Services")
        st.write("• Support Services")

        st.header("🧠 Explainable AI Features")
        st.write("**🔍 LIME Explanations:**")
        st.write("• Word-level importance scoring")
        st.write("• Visual explanation charts")
        st.write("• Interactive text highlighting")
        st.write("• Natural language summaries")

        st.write("**Intent Recognition:** 77 intents")
        st.write("**Urgency Detection:** Low/Medium/High")

    # Main content area
    col1, col2 = st.columns([3, 2])

    with col1:
        st.header("💬 Customer Query Analysis")

        # Text input
        user_input = st.text_area(
            "Enter customer query for AI analysis with explanations:",
            placeholder="Example: I urgently need to transfer money to my friend...",
            height=120,
            help="Enter a banking query and get AI predictions with detailed explanations"
        )

        # Explanation options
        col_opt1, col_opt2 = st.columns(2)
        with col_opt1:
            show_explanations = st.checkbox("Show Detailed Explanations", value=True)
        with col_opt2:
            show_charts = st.checkbox("Show Explanation Charts", value=True)

        # Analysis button
        if st.button("🔍 Analyze Query", type="primary", use_container_width=True):
            if user_input.strip():
                with st.spinner('Analyzing query and generating explanations...'):
                    # Make prediction with explanations
                    result = make_prediction_with_explanations(
                        user_input, model, tokenizer, label_encoders,
                        model_info['max_length'], explainer
                    )

                if result['success']:
                    st.success("Analysis completed with explanations!")

                    # Display basic predictions
                    st.subheader(" AI Predictions")
                    col_cat, col_intent, col_urgency = st.columns(3)

                    with col_cat:
                        confidence = result['category']['confidence']
                        conf_class = get_confidence_class(confidence)
                        st.markdown('<h4>📂 Category</h4>', unsafe_allow_html=True)
                        st.markdown(f'<p style="font-size:18px; font-weight:bold;">{result["category"]["label"]}</p>', unsafe_allow_html=True)
                        st.markdown(f'<span class="{conf_class}">Confidence: {confidence:.1%}</span>', unsafe_allow_html=True)

                    with col_intent:
                        confidence = result['intent']['confidence']
                        conf_class = get_confidence_class(confidence)
                        st.markdown('<h4>🎯 Intent</h4>', unsafe_allow_html=True)
                        st.markdown(f'<p style="font-size:18px; font-weight:bold;">{result["intent"]["label"]}</p>', unsafe_allow_html=True)
                        st.markdown(f'<span class="{conf_class}">Confidence: {confidence:.1%}</span>', unsafe_allow_html=True)

                    with col_urgency:
                        confidence = result['urgency']['confidence']
                        conf_class = get_confidence_class(confidence)
                        urgency_emoji = get_urgency_emoji(result['urgency']['label'])
                        st.markdown('<h4>⚡ Urgency</h4>', unsafe_allow_html=True)
                        st.markdown(f'<p style="font-size:18px; font-weight:bold;">{urgency_emoji} {result["urgency"]["label"]}</p>', unsafe_allow_html=True)
                        st.markdown(f'<span class="{conf_class}">Confidence: {confidence:.1%}</span>', unsafe_allow_html=True)

                    if show_explanations:
                        # Explanation summary - FIXED VERSION
                        st.markdown("---")
                        st.subheader("🧠 AI Explanation Summary")

                        # Extract explanation data
                        category = result['category']['label']
                        intent = result['intent']['label']
                        urgency = result['urgency']['label']

                        cat_words = [word for word, _ in result['explanations']['category'].as_list()[:3]]
                        urgency_words = [word for word, _ in result['explanations']['urgency'].as_list()[:3]]



                        # Display explanation
                        st.info(f"""
                         **AI Decision Explanation**

                        **Category Classification:** The model classified this as **{category}** primarily based on keywords like *{', '.join(cat_words)}*.

                        **Intent Detection:** The specific intent was identified as **{intent}** based on the overall context and word patterns in your query.

                        **Urgency Assessment:** The urgency level was determined as **{urgency}** by analyzing words such as *{', '.join(urgency_words)}*.

                        💡 The highlighted words below show which parts of your text most influenced each decision.
                        """)

                        # Text highlighting for each head
                        st.subheader("🔍 Word Importance Highlighting")

                        tabs = st.tabs(["📂 Category", "🎯 Intent", "⚡ Urgency"])

                        with tabs[0]:
                            highlighted = highlight_text_with_explanations(
                                user_input, result['explanations']['category']
                            )
                            st.markdown(f"<div style='padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>{highlighted}</div>",
                                      unsafe_allow_html=True)
                            st.caption("🟢 Green: Promotes category prediction | 🔴 Red: Demotes category prediction")

                        with tabs[1]:
                            highlighted = highlight_text_with_explanations(
                                user_input, result['explanations']['intent']
                            )
                            st.markdown(f"<div style='padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>{highlighted}</div>",
                                      unsafe_allow_html=True)
                            st.caption("🟢 Green: Promotes intent prediction | 🔴 Red: Demotes intent prediction")

                        with tabs[2]:
                            highlighted = highlight_text_with_explanations(
                                user_input, result['explanations']['urgency']
                            )
                            st.markdown(f"<div style='padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>{highlighted}</div>",
                                      unsafe_allow_html=True)
                            st.caption("🟢 Green: Promotes urgency prediction | 🔴 Red: Demotes urgency prediction")

                    if show_charts:
                        # Interactive explanation charts
                        st.markdown("---")
                        st.subheader("📊 Interactive Explanation Charts")

                        chart_tabs = st.tabs(["📂 Category Chart", "🎯 Intent Chart", "⚡ Urgency Chart"])

                        with chart_tabs[0]:
                            fig = create_explanation_chart(result['explanations']['category'], "Category")
                            if fig:
                                st.plotly_chart(fig, use_container_width=True)

                        with chart_tabs[1]:
                            fig = create_explanation_chart(result['explanations']['intent'], "Intent")
                            if fig:
                                st.plotly_chart(fig, use_container_width=True)

                        with chart_tabs[2]:
                            fig = create_explanation_chart(result['explanations']['urgency'], "Urgency")
                            if fig:
                                st.plotly_chart(fig, use_container_width=True)

                        # Recommendations
                        #recommendations = generate_recommendations(result)
                        #st.markdown("### 💡 Recommended Actions")
                        #for rec in recommendations:
                            #st.write(f"• {rec}")

                else:
                    st.error(f"❌ Error during analysis: {result['error']}")
            else:
                st.warning("Please enter a customer query to analyze.")

    with col2:
        st.header("📈 Model Performance")

        # Performance metrics
        metrics_data = {
            'Category': 0.87,
            'Intent': 0.90,
            'Urgency': 0.75,
            'Overall': 0.83
        }

        for metric, score in metrics_data.items():
            st.metric(f"{metric} Accuracy", f"{score:.1%}")

        st.markdown("---")

        # Explainability info
        st.header("🧠 Explainability Info")
        st.info("""
        **LIME (Local Interpretable Model-Agnostic Explanations)**

        • Explains individual predictions
        • Shows word-level importance
        • Model-agnostic approach
        • Provides trustworthy insights
        """)

        # Example queries
        st.header("💡 Try These Examples")
        example_queries = [
            "I urgently need to transfer money to my friend",
            "My credit card is not working at the ATM machine",
            "Can you please help me check my account balance?",
            "I want to apply for a new debit card immediately",
            "There's a suspicious transaction on my statement"
        ]

        for i, example in enumerate(example_queries):
            if st.button(f"Example {i+1}", key=f"example_{i}", use_container_width=True):
                st.code(example)

# ============= RUN THE APP =============
if __name__ == "__main__":
    main()

Overwriting explainable_banking_ai_app.py


In [ ]:
# ============= LAUNCH ENHANCED EXPLAINABLE BANKING AI APP =============

import subprocess
import threading
import time
from pyngrok import ngrok

# 1. Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# 2. Setup ngrok authentication
# Replace with your actual ngrok authtoken
ngrok_authtoken = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" # MY NGROK TOKEN====================
ngrok.set_auth_token(ngrok_authtoken)

print("🧠 Setting up Explainable Banking AI application...")
print("="*60)

# 3. Function to run Streamlit
def run_streamlit():
    """Run enhanced Streamlit app with explainability"""
    subprocess.run([
        'streamlit', 'run', 'explainable_banking_ai_app.py',
        '--server.port=8502',  # Different port to avoid conflicts
        '--server.headless=True',
        '--server.enableCORS=False',
        '--server.enableXsrfProtection=False'
    ])

# 4. Start Streamlit in a separate thread
print("🚀 Starting Enhanced Streamlit server with LIME integration...")
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.daemon = True
streamlit_thread.start()

# 5. Wait for Streamlit to initialize (longer wait for LIME loading)
print("⏳ Waiting for Streamlit to start (loading model + LIME explainer)...")
time.sleep(20)  # Extra time for LIME initialization

# 6. Create public tunnel with ngrok
print("🌐 Creating public tunnel for explainable AI app...")
try:
    public_url = ngrok.connect(8502)
    print("\n" + "="*70)
    print("✅ EXPLAINABLE BANKING AI WEB APPLICATION IS LIVE!")
    print("="*70)
    print(f" Public URL: {public_url}")
    print(f" Explainable AI App: {public_url}")
    print("="*70)

    print("\n🎯 EXPLAINABILITY FEATURES:")
    print("   • LIME explanations for every prediction")
    print("   • Word importance highlighting")
    print("   • Interactive explanation charts")
    print("   • Natural language explanation summaries")
    print("   • Enhanced UI/UX for transparency")


    print("\n⚠️  Keep this cell running to maintain the public URL")
    print("🛑 Stop this cell to shut down the explainable AI app")

    # Keep the application running
    try:
        while True:
            time.sleep(30)
            print("🧠 Explainable AI app running... (Enhanced with LIME)")
    except KeyboardInterrupt:
        print("\n🛑 Shutting down explainable AI application...")
        ngrok.disconnect(public_url)
        print("✅ Enhanced application stopped successfully")

except Exception as e:
    print(f"❌ Error starting explainable AI application: {e}")
    print("💡 Troubleshooting tips:")
    print("   - Check if all files exist in the specified paths")
    print("   - Verify your ngrok authtoken is correct")
    print("   - Make sure Google Drive is mounted")
    print("   - Ensure LIME is properly installed")
    print("   - Check if port 8502 is available")

    print("\n🔧 Debug information:")
    print(f"   - Model directory: {'/content/drive/MyDrive/Colab Notebooks/AI_CW_010/Final_Model/'}")
    print(f"   - Required files: model, tokenizer, encoders, model_info")
    print(f"   - LIME installation: Required for explainability")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🧠 Setting up Explainable Banking AI application...
🚀 Starting Enhanced Streamlit server with LIME integration...
⏳ Waiting for Streamlit to start (loading model + LIME explainer)...
🌐 Creating public tunnel for explainable AI app...

✅ EXPLAINABLE BANKING AI WEB APPLICATION IS LIVE!
 Public URL: NgrokTunnel: "https://50a400ac8dbb.ngrok-free.app" -> "http://localhost:8502"
 Explainable AI App: NgrokTunnel: "https://50a400ac8dbb.ngrok-free.app" -> "http://localhost:8502"

🎯 EXPLAINABILITY FEATURES:
   • LIME explanations for every prediction
   • Word importance highlighting
   • Interactive explanation charts
   • Natural language explanation summaries
   • Enhanced UI/UX for transparency

⚠️  Keep this cell running to maintain the public URL
🛑 Stop this cell to shut down the explainable AI app
🧠 Explainable AI app running... (Enhanced with LIME)
🧠 Explainable


🛑 Shutting down explainable AI application...
❌ Error starting explainable AI application: [Errno 104] Connection reset by peer
💡 Troubleshooting tips:
   - Check if all files exist in the specified paths
   - Verify your ngrok authtoken is correct
   - Make sure Google Drive is mounted
   - Ensure LIME is properly installed
   - Check if port 8502 is available

🔧 Debug information:
   - Model directory: /content/drive/MyDrive/Colab Notebooks/AI_CW_010/Final_Model/
   - Required files: model, tokenizer, encoders, model_info
   - LIME installation: Required for explainability
